# Data Processing Notebook

This notebook processes data obtained from various sources, including the Cochrane Library, Pfizer documents, non-plain examples from the Clinical Trials API for the Pfizer documents, and files from Trial Summaries.

In [97]:
import pandas as pd

## Loading Datasets

We have four main datasets divided into training and testing sets, each containing plain (pls) and non-plain (non_pls) texts. These datasets are loaded from CSV files for further processing and analysis. The data has undergone data augmentation by paragraphs and has been processed through an API to obtain linguistic features.

- **Dataset 1 (Cochrane Library)**:
  - `data_cochrane_train_non_pls`: Non-plain language training set.
  - `data_cochrane_train_pls`: Plain language training set.
  - `data_cochrane_test_non_pls`: Non-plain language testing set.
  - `data_cochrane_test_pls`: Plain language testing set.

- **Dataset 2 (Pfizer and Clinical Trials)**:
  - `data_pfizer_train_non_pls`: Non-plain language training set.
  - `data_pfizer_train_pls`: Plain language training set.
  - `data_pfizer_test_non_pls`: Non-plain language testing set.
  - `data_pfizer_test_pls`: Plain language testing set.

- **Dataset 3 (Trial Summaries)**:
  - `data_trial_summaries_test_pls`: Plain language testing set.

In [98]:
# Load Dataset 1 (Cochrane Library) training and testing sets
data_cochrane_train_non_pls = pd.read_csv('data/data_cochrane_train_non_pls.csv')
data_cochrane_train_pls = pd.read_csv('data/data_cochrane_train_pls.csv')
data_cochrane_test_non_pls = pd.read_csv('data/data_cochrane_test_non_pls.csv')
data_cochrane_test_pls = pd.read_csv('data/data_cochrane_test_pls.csv')

# Load Dataset 2 (Pfizer and Clinical Trials) training and testing sets
data_pfizer_train_non_pls = pd.read_csv('data/data_pfizer_train_non_pls.csv')
data_pfizer_train_pls = pd.read_csv('data/data_pfizer_train_pls.csv')
data_pfizer_test_non_pls = pd.read_csv('data/data_pfizer_test_non_pls.csv')
data_pfizer_test_pls = pd.read_csv('data/data_pfizer_test_pls.csv')

# Load Dataset 3 (Trial Summaries) only test set
data_trial_summaries_test_pls = pd.read_csv('data/data_trial_summaries_test_pls.csv')

## Labeling Data

To facilitate the identification and classification of plain and non-plain texts, we add a `label` column to each dataset:
- A label of `0` is assigned to non-plain texts.
- A label of `1` is assigned to plain texts.

In [99]:
# Add a label column to Dataset 1 (Cochrane Library) training sets
data_cochrane_train_non_pls['label'] = 0
data_cochrane_train_pls['label'] = 1

# Add a label column to Dataset 2 (Pfizer and Clinical Trials) training sets
data_pfizer_train_non_pls['label'] = 0
data_pfizer_train_pls['label'] = 1

# Add a label column to Dataset 1 (Cochrane Library) testing sets
data_cochrane_test_non_pls['label'] = 0
data_cochrane_test_pls['label'] = 1

# Add a label column to Dataset 2 (Pfizer and Clinical Trials) testing sets
data_pfizer_test_non_pls['label'] = 0
data_pfizer_test_pls['label'] = 1

# Add a label column to Dataset 3 (TS)
data_trial_summaries_test_pls['label'] = 1

## Combining and Shuffling Data

To create comprehensive training and testing datasets, we concatenate the individual datasets (both plain and non-plain texts) into single DataFrames. This helps ensure that the machine learning models have a diverse and representative set of examples to learn from.

After concatenating, we shuffle the data to remove any order bias and ensure a random distribution of samples. On the other hand, the data from TS remains separated.

In [100]:
# Concatenate training data from both datasets
data_train = pd.concat([data_cochrane_train_non_pls, data_cochrane_train_pls, data_pfizer_train_non_pls, data_pfizer_train_pls])

# Concatenate testing data from both datasets
data_test = pd.concat([data_cochrane_test_non_pls, data_cochrane_test_pls, data_pfizer_test_non_pls, data_pfizer_test_pls])

# Shuffle the training data to ensure random distribution
data_train = data_train.sample(frac=1).reset_index(drop=True)

# Shuffle the testing data to ensure random distribution
data_test = data_test.sample(frac=1).reset_index(drop=True)
data_trial_summaries_test_pls = data_trial_summaries_test_pls.sample(frac=1).reset_index(drop=True)

## Dropping columns
### Dropping Zero-Value columns
We identify and remove columns that contain only zero values to streamline the datasets.

1. Identify columns with only zero values.
2. Drop these columns from the training and testing sets.

In [101]:
# Identify columns in the training data that contain only zero values
zeros = data_train.columns[(data_train == 0).all()]
print(zeros)

Index(['conjunctions'], dtype='object')


### Dropping Columns With a Small Non-Zero Percentage

We calculate the percentage of non-zero values for each column in the training dataset. This helps in understanding the distribution of values and identifying columns that might have a significant amount of zero values.

In [102]:
mask = (data_train != 0)
percent_zeros = mask.sum() / mask.count() * 100
percent_zeros.to_frame()

,0
file,100.000000
Kincaid,100.000000
ARI,100.000000
Coleman-Liau,100.000000
FleschReadingEase,100.000000
...,...
tobeverb,99.971117
auxverb,82.705123
conjunction,99.995874
nominalization,99.971117


Next, we set a threshold of 3% to identify columns with a low percentage of non-zero values. Columns below this threshold will be listed for removal.

In [103]:
# Set a threshold for the minimum acceptable percentage of non-zero values
threshold = 3

# Identify columns that fall below the threshold
columns_to_drop = percent_zeros[percent_zeros < threshold].index

# List the columns to be dropped
columns_to_drop

Index(['conjunctions', 'events', 'languages'], dtype='object')

### Dropping Redundant Columns

We identify some columns that are unnecessary since their information is already found in others.

In [104]:
# Add to columns to drop
columns_to_drop = columns_to_drop.append(pd.Index(['characters', 'sentences', 'words', 'paragraphs']))
columns_to_drop = columns_to_drop.append(zeros)

In [105]:
columns_to_drop

Index(['conjunctions', 'events', 'languages', 'characters', 'sentences',
       'words', 'paragraphs', 'conjunctions'],
      dtype='object')

### Dropping Columns With Not Enough _p_-value

Taking into account the hypothesis tests carried out, we eliminated the columns with a low p-value.

In [106]:
columns_to_drop = columns_to_drop.append(pd.Index(['money', 'passive_voice', 'interjections', 'facilities']))
columns_to_drop

Index(['conjunctions', 'events', 'languages', 'characters', 'sentences',
       'words', 'paragraphs', 'conjunctions', 'money', 'passive_voice',
       'interjections', 'facilities'],
      dtype='object')

In [107]:
# Drop the columns with low non-zero percentage from the training and testing dataset
data_train = data_train.drop(columns_to_drop, axis=1)
data_test = data_test.drop(columns_to_drop, axis=1)
data_trial_summaries_test_pls = data_trial_summaries_test_pls.drop(columns_to_drop, axis=1)

Then, we define our feature set (`X`) and label set (`y`)

In [108]:
X_train = data_train.drop(['label'], axis=1)
y_train = data_train['label']

X_test = data_test.drop(['label'], axis=1)
y_test = data_test['label']

X_ts = data_trial_summaries_test_pls.drop(['label'], axis=1)
y_ts = data_trial_summaries_test_pls['label']

## Normalization of Data

In this section, we normalize the data to ensure that all features are on a comparable scale. We have identified that certain columns require a specific normalization method based on min-max normalization, while other columns will be normalized by the count of words of their respective type.

### Special Columns

The following columns require min-max normalization:

- `total_words`
- `total_sentences`
- `total_characters`
- `characters_per_word`
- `syll_per_word`
- `words_per_sentence`
- `sentences_per_paragraph`
- `type_token_ratio`
- `syllables`
- `wordtypes`
- `Kincaid`
- `ARI`
- `Coleman-Liau`
- `FleschReadingEase`
- `GunningFogIndex`
- `LIX`
- `SMOGIndex`
- `RIX`
- `DaleChallIndex`

In [109]:
# Columns that require min-max normalization
special_columns = ['total_words', 'total_sentences', 'total_characters',                 
                   'characters_per_word','syll_per_word', 'words_per_sentence', 'sentences_per_paragraph',
                   'type_token_ratio', 'syllables', 'wordtypes', 'Kincaid', 'ARI', 'Coleman-Liau', 'FleschReadingEase',
                   'GunningFogIndex', 'LIX', 'SMOGIndex', 'RIX', 'DaleChallIndex']

Columns to ignore and we don't want to normalize.

In [110]:
cols_to_ignore = ['file', 'text']

We normalize the columns not listed in the special columns by dividing their values by the `total_words` column. This ensures these features are scaled appropriately relative to the total word count.

In [111]:
# Normalize columns not listed in special columns by dividing by total_words
for column in X_train.columns:
    if column not in special_columns:
        if column in cols_to_ignore:
            continue
        X_train[column] = X_train[column] / X_train['total_words']
        X_test[column] = X_test[column] / X_test['total_words']
        X_ts[column] = X_ts[column] / X_ts['total_words']

### Min-Max Normalization for Special Columns

For columns identified as requiring min-max normalization, we scale their values to a range between 0 and 1.

In [112]:
# Initialize dictionaries to store the minimum and maximum values for each special column
maximos = {}
minimos = {}

# Perform min-max normalization for each special column
for column in special_columns:
    if column in cols_to_ignore:
        continue
    # Calculate the minimum and maximum values across training and testing datasets
    minimum = min(X_train[column].min(), X_test[column].min())
    maximum = max(X_train[column].max(), X_test[column].max())
    
    # Store the calculated minimum and maximum values
    maximos[str(column)] = maximum
    minimos[str(column)] = minimum

    # Apply min-max normalization to the training and testing datasets
    X_train[column] = (X_train[column] - minimum) / (maximum - minimum)
    X_test[column] = (X_test[column] - minimum) / (maximum - minimum)
    X_ts[column] = (X_ts[column] - minimum) / (maximum - minimum)


## Combining Normalized Data

In this section, we combine the normalized training and testing datasets back into a single DataFrame for further analysis. We also reattach the labels to the combined dataset.

### Steps:
1. Concatenate the normalized training and testing feature sets.
2. Reattach the labels to the combined dataset.

In [113]:
# Combine the normalized training and testing feature sets into a single DataFrame
data = pd.concat([X_train, X_test])

# Reattach the labels to the combined dataset
data['label'] = pd.concat([y_train, y_test])

# Reattach the labels to the combined dataset for TS dataset.
data_ts = pd.concat([X_ts])
data_ts['label'] = y_ts

In [114]:
data

,file,Kincaid,ARI,Coleman-Liau,FleschReadingEase,GunningFogIndex,LIX,SMOGIndex,RIX,DaleChallIndex,...,syllables,wordtypes,long_words,complex_words,complex_words_dc,tobeverb,auxverb,conjunction,nominalization,label
0,10.1002-14651858.CD013192-pls.txt_accumulated_...,0.048432,0.038289,0.622900,0.918237,0.043913,0.045974,0.128116,0.035485,0.063285,...,0.078577,0.252727,0.296729,0.198598,0.380841,0.044393,0.021028,0.035047,0.014019,1
1,10.1002-14651858.CD011260.pub2-pls.txt,0.037910,0.028528,0.596640,0.931635,0.037341,0.036836,0.115226,0.026766,0.081336,...,0.085409,0.283636,0.291139,0.215190,0.453586,0.035865,0.018987,0.037975,0.033755,1
2,10.1002-14651858.CD005998.pub2-abstract.txt_ac...,0.064215,0.055382,0.730536,0.896475,0.069792,0.068733,0.213586,0.063197,0.150738,...,0.173808,0.454545,0.395319,0.319896,0.628088,0.028609,0.007802,0.049415,0.071521,0
3,10.1002-14651858.CD010750.pub2-abstract.txt_ac...,0.055244,0.048598,0.646183,0.916090,0.054629,0.057442,0.154931,0.047469,0.093017,...,0.115018,0.418182,0.321070,0.214047,0.454849,0.035117,0.000000,0.026756,0.035117,0
4,10.1002-14651858.CD010985.pub4-abstract.txt,0.033381,0.025015,0.807795,0.918596,0.024372,0.034230,0.075834,0.020074,0.092505,...,0.054093,0.145455,0.380137,0.222603,0.513699,0.023973,0.003425,0.044521,0.034247,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11844,10.1002-14651858.CD011451.pub2-abstract.txt,0.068927,0.059567,0.672965,0.895746,0.071115,0.069480,0.206742,0.061214,0.159439,...,0.176370,0.503636,0.360677,0.286458,0.669271,0.022135,0.002604,0.040365,0.031250,0
11845,10.1002-14651858.CD004945.pub5-pls.txt_accumul...,0.033351,0.025607,0.560981,0.941870,0.031568,0.028060,0.094681,0.018587,0.069928,...,0.037011,0.161818,0.228261,0.177536,0.420290,0.028986,0.018116,0.057971,0.036232,1
11846,10.1002-14651858.CD003091.pub4-pls.txt_accumul...,0.049534,0.040519,0.669550,0.915306,0.046569,0.052705,0.139192,0.043548,0.068412,...,0.043416,0.125455,0.350365,0.218978,0.394161,0.040146,0.003650,0.029197,0.047445,1
11847,10.1002-14651858.CD012089.pub2-abstract.txt_ac...,0.066634,0.059266,0.746103,0.895460,0.060119,0.068894,0.164765,0.062268,0.096665,...,0.182064,0.514545,0.369458,0.210591,0.453202,0.019704,0.001232,0.045567,0.075123,0


In [115]:
data_ts

,file,Kincaid,ARI,Coleman-Liau,FleschReadingEase,GunningFogIndex,LIX,SMOGIndex,RIX,DaleChallIndex,...,syllables,wordtypes,long_words,complex_words,complex_words_dc,tobeverb,auxverb,conjunction,nominalization,label
0,11433-MT_MED_9011_page5.txt,0.016401,0.013017,0.549351,0.965476,0.009439,0.010428,0.025849,0.006196,0.029485,...,0.012811,0.038182,0.186441,0.101695,0.327684,0.005650,0.005650,0.033898,0.016949,1
1,11439-MT_MED_9011_page7.txt,0.006421,0.003015,0.478285,0.977831,0.002170,0.016611,0.006829,0.006148,0.076018,...,0.037438,0.085455,0.289634,0.106707,0.460366,0.036585,0.015244,0.015244,0.060976,1
2,11409-MT_MED_9011_page1.txt,0.015392,0.011102,0.527759,0.966386,0.010147,0.011110,0.030174,0.006506,0.036877,...,0.022064,0.074545,0.200893,0.116071,0.352679,0.017857,0.000000,0.035714,0.013393,1
3,11283-MT_MED_9011_page6.txt,0.022580,0.018196,0.573247,0.956490,0.016690,0.029770,0.047978,0.019445,0.053637,...,0.040000,0.080000,0.309211,0.125000,0.391447,0.042763,0.016447,0.019737,0.026316,1
4,11291-MT_MED_9011_page9.txt,0.018978,0.019244,0.703737,0.956410,0.009010,0.021175,0.025849,0.012082,0.050689,...,0.040427,0.176364,0.269231,0.102564,0.375000,0.032051,0.009615,0.032051,0.028846,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,11056-MT_MED_9011_page7.txt,0.010041,0.007735,0.508416,0.975886,0.012188,0.015536,0.038350,0.008055,0.059848,...,-0.003559,-0.058182,0.252525,0.151515,0.414141,0.040404,0.000000,0.010101,0.000000,1
1030,11288-MT_MED_9011_page5.txt,0.018231,0.012191,0.569233,0.956804,0.012817,0.017801,0.040307,0.009913,0.014472,...,0.023060,0.060000,0.262911,0.145540,0.291080,0.032864,0.023474,0.028169,0.028169,1
1031,10967-MT_MED_9011_page6.txt,0.012652,0.011579,0.569581,0.970859,0.009480,0.022223,0.029323,0.011896,0.038670,...,0.027473,0.054545,0.302326,0.124031,0.360465,0.034884,0.015504,0.011628,0.023256,1
1032,1017-MT_MED_9011_page1.txt,0.021544,0.017734,0.654761,0.950910,0.012580,0.020625,0.038350,0.012082,0.038101,...,0.011815,0.029091,0.269231,0.128205,0.352564,0.006410,0.000000,0.032051,0.006410,1


## Saving the Data

After normalizing and combining the data, we save the cleaned and processed datasets into CSV files for future use. This includes saving the training and testing datasets, as well as separating the plain and non-plain test sets.

In [116]:
# Combine the training features and labels, then save to data_train.csv
data = pd.concat([X_train, y_train], axis=1)
data.to_csv('data_clean/data_train.csv', index=False)

# Combine the testing features and labels, then save to data_test.csv
data = pd.concat([X_test, y_test], axis=1)
data.to_csv('data_clean/data_test.csv', index=False)

data_ts.to_csv('data_clean/data_ts.csv', index=False)